# Appendix 10.1: Chaining Prompts

- [Lesson](#lesson)


## 设置
运行以下设置单元格以加载您的 API 密钥并建立 get_completion 辅助函数。

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

# Has been rewritten to take in a messages list of arbitrary length
def get_completion(messages, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=messages
    )
    return message.content[0].text

---

## 课程

俗话说，"写作就是重写。"事实证明，**当被要求这样做时，LLM 通常可以提高其回答的准确性！**
有很多方法可以提示 Claude "重新思考"。那些让人类重新检查工作感觉很自然的方法通常对 Claude 也有效。（查看我们的提示链接文档以获取更多关于何时以及如何使用提示链接的示例。） [prompt chaining documentation](https://docs.anthropic.com/claude/docs/chain-prompts) for further examples of when and how to use prompt chaining.)

### 例子

在这个例子中，我们让 LLM 想出十个单词……但其中一个或多个并不是真正的单词。

In [2]:
# Initial prompt
first_user = "Name ten words that all end with the exact letters 'ab'."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Fab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Zab


**要求 Claude 让它的答案更准确修复了这个错误！**
下面，我们从上面提取了 Claude 的错误回答，并在对话中添加了另一轮，要求 Claude 修正它之前的答案。

In [3]:
second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Fab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Zab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are 10 real words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab


但是LLM 修正答案仅仅是因为我们告诉它这样做吗？如果我们一开始就有一个正确的答案呢？LLM 会失去信心吗？在这里，我们在 first_response 的位置放置了一个正确的回答，并要求它再次检查。

In [4]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words."}]

------------------------------------- Claude's response -------------------------------------
Here are 10 real words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab


你可能会注意到，如果你从上面的代码块生成几次回答，Claude 大多数时候会保持单词不变，但仍然偶尔会改变单词，即使它们都已经是正确的。我们能做什么来缓解这个问题呢？根据第8章，我们可以给 LLM 一个回避选项！让我们再试一次。

In [5]:
first_user = "Name ten words that all end with the exact letters 'ab'."

first_response = """Here are 10 words that end with the letters 'ab':

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab"""

second_user = "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': "Name ten words that all end with the exact letters 'ab'."}, {'role': 'assistant', 'content': "Here are 10 words that end with the letters 'ab':\n\n1. Cab\n2. Dab\n3. Grab\n4. Gab\n5. Jab\n6. Lab\n7. Nab\n8. Slab\n9. Tab\n10. Blab"}, {'role': 'user', 'content': "Please find replacements for all 'words' that are not real words. If all the words are real words, return the original list."}]

------------------------------------- Claude's response -------------------------------------
Okay, here is the list with replacements for any words that are not real:

1. Cab
2. Dab
3. Grab
4. Gab
5. Jab
6. Lab
7. Nab
8. Slab
9. Tab
10. Blab

All of these are real words, so the original list is returned.


尝试从上面的代码生成几次回答，看看 LLM 现在在坚持自己的观点方面做得更好了。
你也可以使用 **提示链接来要求 LLM 改进它的回答**。下面，我们让 LLM 先写一个故事，然后改进它写的故事。你的个人品味可能不同，但许多人可能会同意 LLM 的第二个版本更好。
首先，让我们生成 LLM 故事的第一个版本。

In [6]:
# Initial prompt
first_user = "Write a three-sentence short story about a girl who likes to run."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print(first_response)

Here is a three-sentence short story about a girl who likes to run:

Amelia loved the feeling of the wind rushing through her hair as she raced down the trail, her feet pounding against the soft earth. With each stride, she felt a sense of freedom and exhilaration that she couldn't find anywhere else. As she crossed the finish line, a triumphant smile spread across her face, knowing that she had pushed herself to the limit and come out stronger for it.


现在，让LLM改进第一版草稿

In [7]:
second_user = "Make the story better."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Write a three-sentence short story about a girl who likes to run.'}, {'role': 'assistant', 'content': "Here is a three-sentence short story about a girl who likes to run:\n\nAmelia loved the feeling of the wind rushing through her hair as she raced down the trail, her feet pounding against the soft earth. With each stride, she felt a sense of freedom and exhilaration that she couldn't find anywhere else. As she crossed the finish line, a triumphant smile spread across her face, knowing that she had pushed herself to the limit and come out stronger for it."}, {'role': 'user', 'content': 'Make the story better.'}]

------------------------------------- Claude's response -------------------------------------
Here is the three-sentence short story with some improvements:

Amelia's heart raced as she pushed herself harder, her legs pumping with each stride as she fl

这种替换形式非常强大。我们一直在使用替换占位符来传入列表、单词、LLM之前的回答等等。你也 **可以使用替换来做我们称之为"function calling"的事情，即要求 LLM 执行某个函数，然后获取该函数的结果，并要求 LLM 用这些结果做更多的事情**。它的工作原理与任何其他替换一样。在下一个附录中会有更多相关内容。
下面是另一个将 LLM 一次调用的结果插入到另一个更长调用中的示例。让我们从第一个提示开始（这次包括预填充 Claude 的回答）。

In [9]:
first_user = """Find all names from the below text:

"Hey, Jesse. It's me, Erin. I'm calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too."""

prefill = "<names>"

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill
    
    }
]

# Store and print Claude's response
first_response = get_completion(messages)
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(first_response)

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>'}]

------------------------------------- Claude's response -------------------------------------

Jesse
Erin
Joey
Keisha
Mel
</names>


然后把这个名字列表传给提示词

In [10]:
second_user = "Alphabetize the list."

# API messages array
messages = [
    {
        "role": "user",
        "content": first_user
    
    },
    {
        "role": "assistant",
        "content": prefill + "\n" + first_response
    
    },
    {
        "role": "user",
        "content": second_user
    
    }
]

# Print Claude's response
print("------------------------ Full messsages array with variable substutions ------------------------")
print(messages)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(messages))

------------------------ Full messsages array with variable substutions ------------------------
[{'role': 'user', 'content': 'Find all names from the below text:\n\n"Hey, Jesse. It\'s me, Erin. I\'m calling about the party that Joey is throwing tomorrow. Keisha said she would come and I think Mel will be there too.'}, {'role': 'assistant', 'content': '<names>\n\nJesse\nErin\nJoey\nKeisha\nMel\n</names>'}, {'role': 'user', 'content': 'Alphabetize the list.'}]

------------------------------------- Claude's response -------------------------------------
<names>
Erin
Jesse
Joey
Keisha
Mel
</names>


现在你已经了解了提示链接，请前往附录 10.2 学习如何使用提示链接实现函数调用。